In [1]:
%matplotlib qt

from math import *
from numpy import *
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d
import matplotlib.patheffects as path_effects
import xlsxwriter








In [2]:
#Choix du type de maison

type = 1 #[0] plat, [1] un pan, [2] deux_pans, [4] quatre_pans

"""Ajouter un menu pour choisir l'emplacement etc"""


"Ajouter un menu pour choisir l'emplacement etc"

In [3]:
"""Définition des dimensions"""
#Toutes les longueurs sont données en m
#Longueur des murs de la maison, suivant l'axe du faitage, ou suivant la plus grande dimension pour un toit à 4 pans
L = 15

#largeur des murs, ou plus petite dimension
l = 10

#Hauteur du faitage ou hauteur max de la toiture pour un toit plat
h_fait = 8.5

#Angle de toiture, en degrés
alpha = 21.8

#POUR UN TOIT PLAT, hauteur des murs
h_mur = 6

#Débords de toiture sur les pignons
debord_p1 = 0.5
debord_p2 = 0.5

#Débords de toiture sur les longs pans
debord_l1 = 0
debord_l2 = 0.5


In [4]:
#Permet de définir une fonction pour avoir des flèches en 3D

class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)


In [5]:
def point_sur_toit(a):
    if type == 1 or type == 0: #toit un pan ou plat
        z = h_fait-(a[0]+debord_l1)*tan(radians(alpha))
    return [a[0],a[1],z]

In [6]:
def point_sur_mur(a):
    if type == 0:
        return [a[0],a[1],h_mur]
    else :
        return point_sur_toit(a)

In [7]:
liste_couleurs = ['blue','lime','orange','blueviolet','brown','red','forestgreen','yellow','cyan']
liste_noms = ['A','B','C','D','E','F','G','H','I']



In [8]:
def calcul_distance(a,b):
    return sqrt((a[0]-b[0])**2+(a[1]-b[1])**2+(a[2]-b[2])**2)

In [9]:
def calcul_surface_polygone(sommets): #calcul de la surface à partir de la liste de ses sommets, à condition qu'ils soient dans l'ordre
    n = len(sommets) #un polygone à n sommets sera decomposé en n-2 triangles
    
    surface = 0 #initialisation de la surface
    
    for i in range (n-2):  #decomposition en triangles
        surface += calcul_surface_triangle([sommets[0]]+sommets[i+1:i+3])
    
    return surface

In [10]:
def calcul_surface_triangle(sommets): #calcul de la surface d'un triangle à partir de la liste de ses sommets
    
    a,b,c=sommets
    
    ab = calcul_distance(a,b)
    ac = calcul_distance(a,c)
    bc = calcul_distance(b,c)
    
    p = (ab+bc+ac)/2
    surface = sqrt(p*(p-ab)*(p-bc)*(p-ac))
    
    return surface

In [11]:
#définition des points qui définissent la maison

    
#points au sol, au niveau 0 (dalle)
O = fond1 = [0,0,0]
fond2 = [l,0,0]
fond3 = [l,L,0]
fond4 = [0,L,0]

#au sommet des murs
mur1,mur2,mur3,mur4 = point_sur_mur(fond1),point_sur_mur(fond2),point_sur_mur(fond3),point_sur_mur(fond4)

#sur le faitage
fait1 = [-debord_l1,-debord_p1,h_fait]
fait4 = [-debord_l1,L+debord_p2,h_fait]

#pour l'egout
eg1 = point_sur_toit([-debord_l1,-debord_p1,0])
eg2 = point_sur_toit([l+debord_l2,-debord_p1,0])
eg3 = point_sur_toit([l+debord_l2,L+debord_p2,0]) #pour un toit 1 pan, eg1 = fait1 et eg4=fait4, mais les eg sont utiles pour les autres modeles, dans la fonction etude_toit
eg4 = point_sur_toit([-debord_l1,L+debord_p2,0])

if type !=0:
    h_mur = h_fait


In [12]:
def etude_murs(direction,liste_surfaces): #4 directions possibles : suivant '+x', '-x', '+y', '-y'
    
    if direction == '+x':
        vent = np.array([1,0,0])
        origines = [fond1,fond4,fond2,fond3]
    elif direction == '-x':
        vent = np.array([-1,0,0])
        origines = [fond2,fond3,fond1,fond4]
    elif direction == '+y':
        vent = np.array([0,1,0])
        origines = [fond1,fond2,fond3,fond4]
    else :
        vent = np.array([0,-1,0])
        origines = [fond4,fond3,fond2,fond1]
    
    if direction == '+x' or direction == '-x':
        b,d = L,l
    else :
        b,d = l,L
    
        
    e=min(b,2*h_mur)
    
    if e<d :
        L_A = e/5
        L_B = 4/5*e
        L_C = d-e
        
    elif e<(5*d):
        L_A = e/5
        L_B = d-e/5
        L_C = 0
    
    else :
        L_A = D
        L_B = 0
        L_C = 0
    L_j=[L_A,L_B,L_C]
        
    surfaces = [] 
    centres = []
    
    for i in range(2):
        origine = origines[i]
        for j in range (3) : # de A à C
            point_suivant = (np.array(origine)+vent*L_j[j]).tolist()
            
            if type == 0:
                point3,point4=point_suivant.copy(),origine.copy()
                point3[2]=h_mur
                point4[2]=h_mur
                
                
            elif type == 1:
                point3 = point_sur_mur(point_suivant)
                point4 = point_sur_mur(origine)
            
            liste_surfaces.append([liste_noms[j]+str(i+1),liste_couleurs[j],origine,point_suivant,point3,point4])
            
            origine=point_suivant
            
    #surfaces D et E    
    liste_surfaces.append([liste_noms[3],liste_couleurs[3],origines[0],point_sur_mur(origines[0]),point_sur_mur(origines[1]),origines[1]])
    liste_surfaces.append([liste_noms[4],liste_couleurs[4],origines[2],point_sur_mur(origines[2]),point_sur_mur(origines[3]),origines[3]])
    
    return liste_surfaces
    

In [13]:
def etude_toit(direction,liste_surfaces):
    

    
    if direction == '+x':
        vent = np.array([1,0,0])
        origines = [eg1]
        progression = np.array([0,1,0])
        
        #origines = [eg1,eg4,eg2,eg3]
    elif direction == '-x':
        vent = np.array([-1,0,0])
        origines = [eg3]
        progression = np.array([0,-1,0])
        
        #origines = [eg2,eg3,eg1,egt4]
    elif direction == '+y':
        vent = np.array([0,1,0])
        origines = [eg2]
        progression = np.array([-1,0,0])
        
        #origines = [eg1,eg2,eg3,eg4]
    else :
        vent = np.array([0,-1,0])
        origines = [eg4]
        progression = np.array([1,0,0])
        
        #origines = [eg4,eg3,eg2,eg1]
    
    if direction == '+x' or direction == '-x':
        b,d = L+debord_p1+debord_p2,l+debord_l1+debord_l2
    else :
        b,d = l+debord_l1+debord_l2,L+debord_p1+debord_p2
        
    e=min(b,2*h_fait)
    
    L_F = e/10 #F et G sont communs à tous les types de toits
    l_F = e/4
    L_G = L_F
    l_G = b-e/2
        
    if type <=1 : #pour toit plat, 1 pan
        if direction == "+x" or direction == "-x":
            L_H = d-e/10
            L_I = 0
        else :
            L_H = e/2-e/10
            L_I = d-L_H-L_F
            
        l_H = l_I = b

        liste_origines = [origines[0],point_sur_toit(np.array(origines[0])+l_F*progression),point_sur_toit(np.array(origines[0])+(l_F+l_G)*progression),point_sur_toit(np.array(origines[0])+L_F*vent),point_sur_toit(np.array(origines[0])+(L_F+L_H)*vent)] #liste des points origines locales des surfaces F1,G,F2,H,I
        liste_longueurs = [L_F,L_G,L_F,L_H,L_I]
        liste_largeurs = [l_F,l_G,l_F,l_H,l_I]

        liste_indices=[5,6,5,7,8]

        for i in range(5): #F,G,F,H,I
            origine = liste_origines[i]
            longueur = liste_longueurs[i]
            largeur = liste_largeurs[i]
            point2 = point_sur_toit(np.array(origine)+largeur*progression)
            point3 = point_sur_toit(np.array(point2)+vent*longueur)
            point4 = point_sur_toit(np.array(origine)+longueur*vent)

            j = liste_indices[i]

            liste_surfaces.append([liste_noms[j],liste_couleurs[j],origine,point2,point3,point4])
    elif type == 2 : #toit 2 pans
        if direction == '+x' or direction == '-x':
            if direction == '+x':
                L_H=l/2+debord_l1-e/10
            else :
                L_H=l/2+debord_l2-e/10
                
            l_h=b
            L_J=e/10   
            l_j=b
            L_I=d-L_J-L_F
            l_I=b
        else :

            L_H=e/2-e/10 ##################il faut differencier +y et -y
            L_I=d-L_F-L_H
            l_G=0
            l_H=0
            L_J=l_j=0
                
        
            
    return liste_surfaces

In [14]:
def centre_de_gravite(sommets):#centre de gravité d'un polygone plan
    
    S = 0
    somme_xGi_Si = 0
    somme_yGi_Si = 0
    somme_zGi_Si = 0
    
    n=len(sommets)
      
    for i in range(n-2):#on divise en n-2 triangles
        sommets_triangle = [sommets[0]]+sommets[i+1:i+3]
        a,b,c=sommets_triangle
        xGi = np.average([a[0],b[0],c[0]])
        yGi = np.average([a[1],b[1],c[1]])
        zGi = np.average([a[2],b[2],c[2]])
        Si = calcul_surface_triangle(sommets_triangle)
        
        S += Si
        somme_xGi_Si += xGi * Si
        somme_yGi_Si += yGi * Si
        somme_zGi_Si += zGi * Si
    if S == 0 :
        return [0,0,0]
    
    xG = round(somme_xGi_Si/S,3)
    yG = round(somme_yGi_Si/S,3)
    zG = round(somme_zGi_Si/S,3)
    
    return [xG,yG,zG]
        

In [15]:
def analyse_surfaces(liste_surfaces):
    for s in liste_surfaces :
        s.append(centre_de_gravite(s[2:6]))
        s.append(calcul_surface_polygone(s[2:6]))
    return liste_surfaces

In [16]:
def analyse_complete(direction,graphique,excel):

    with pd.ExcelWriter('export.xlsx',engine='xlsxwriter') as writer:
        for direction in ['+x','-x','+y','-y']:
            liste_surfaces = [] #liste des surfaces remarquables, qui contiendra [[nom,couleur,point1,point2,point3,point4,G,surface],[...],...]
            liste_surfaces = etude_murs(direction,liste_surfaces)
            liste_surfaces = etude_toit(direction,liste_surfaces)
            liste_surfaces = analyse_surfaces(liste_surfaces)


            image='image'+direction+'.png'

            if graphique:
                plt = representation_graphique(liste_surfaces,direction)
                
                plt.show()

            df = pd.DataFrame(liste_surfaces,columns=['Nom de la surface','Couleur','P1','P2','P3','P4','Centre de gravité','Aire (m²)'])
            df = df[['Nom de la surface','Aire (m²)','Centre de gravité']]
            df = df.set_index('Nom de la surface')

            if excel :
                
                df.to_excel(writer, sheet_name = direction)
                workbook  = writer.book
                worksheet = writer.sheets[direction]
                format1 = workbook.add_format({'num_format': '#,##0.00'})
                format1.set_align('center')
                worksheet.set_column('A:C', 18, format1)
                
                if graphique:
                    plt.savefig(image)
                    col = df.shape[1]+2 #indice de la première colonne libre+1
                    
                    worksheet.insert_image(0,col, image,{'x_scale': 1, 'y_scale': 1}) 
                    
                
    
   

    

In [17]:
def representation_graphique(liste_surfaces,direction):
    
    def dessin_polygone(a,b,c,d,color):
        polygone = [[a,b,c,d]]
        ax.add_collection3d(Poly3DCollection(polygone,facecolors=color,edgecolors='black',linewidths=2,antialiased=False,alpha=0.25))
    
    fig = plt.figure()
    fig.set_size_inches(8,6)
    ax = Axes3D(fig)

    for s in liste_surfaces:
        dessin_polygone(s[2],s[3],s[4],s[5],s[1])
        
    for s in liste_surfaces :
        label = str(s[0])+'\nS = '+str(round(s[7],2))+'m²\n'+str(s[6])
        texte = ax.text(s[6][0],s[6][1],s[6][2],label,color=s[1],multialignment='center',horizontalalignment='center',verticalalignment='center',weight = 'bold')
        texte.set_path_effects([path_effects.Stroke(linewidth=1, foreground='black'),path_effects.Normal()])
    
    if direction == '+x':
        xfleche = [-5,-0.5]
        yfleche = [L/2,L/2]
    elif direction == '-x':
        xfleche = [l+5,l+0.5]
        yfleche = [L/2,L/2]
    elif direction == '+y':
        xfleche = [l/2,l/2]
        yfleche = [-5,-0.5]
    elif direction == '-y':
        xfleche = [l/2,l/2]
        yfleche = [L+5,L+0.5]
    
    #flèche qui représente le sens du vent
    fleche_vent = Arrow3D(xfleche, yfleche, [h_fait/2, h_fait/2], mutation_scale=50, arrowstyle='-|>', color="r",lw=3)
    fleche_vent.set_arrowstyle('simple',tail_width=0.2)
    ax.add_artist(fleche_vent)
    
    
    ax.set_xlabel('axe X')
    ax.set_ylabel('axe Y')
    ax.set_zlabel('axe Z')
    ax.set_xlim3d(-debord_l1 ,l+debord_l2 )
    ax.set_ylim3d(-debord_p1 ,L+debord_p2 )
    ax.set_zlim3d(0,max(h_fait,h_mur))

    return plt
    



In [ ]:
analyse_complete('+x',True,True) #direction du vent : '+x', '-x', '+y', '-y' ; Représentation graphique : True/False ; Export Excel : True/False